In [1]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving ecommerce_customer_data.csv to ecommerce_customer_data.csv


In [2]:
df = pd.read_csv("ecommerce_customer_data.csv")
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Customer ID            250000 non-null  int64  
 1   Purchase Date          250000 non-null  object 
 2   Product Category       250000 non-null  object 
 3   Product Price          250000 non-null  int64  
 4   Quantity               250000 non-null  int64  
 5   Total Purchase Amount  250000 non-null  int64  
 6   Payment Method         250000 non-null  object 
 7   Customer Age           250000 non-null  int64  
 8   Returns                202618 non-null  float64
 9   Customer Name          250000 non-null  object 
 10  Age                    250000 non-null  int64  
 11  Gender                 250000 non-null  object 
 12  Churn                  250000 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 24.8+ MB


,0
Customer ID,0
Purchase Date,0
Product Category,0
Product Price,0
Quantity,0
Total Purchase Amount,0
Payment Method,0
Customer Age,0
Returns,47382
Customer Name,0


In [3]:
df["Returns"] = df["Returns"].fillna(0)

In [4]:
df["Returns"] = df["Returns"].astype(int)

In [5]:
df = df[df["Total Purchase Amount"] > 0]
df = df[df["Quantity"] > 0]

In [15]:
df.to_csv("cleaned_ecommerce_data.csv", index=False)
from google.colab import files
files.download("cleaned_ecommerce_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
df["Purchase Date"] = pd.to_datetime(df["Purchase Date"])
import datetime as dt
reference_date = df["Purchase Date"].max()
reference_date

Timestamp('2023-09-13 18:42:49')

In [8]:
rfm = df.groupby("Customer ID").agg({
    "Purchase Date": lambda x: (reference_date - x.max()).days,
    "Customer ID": "count",
    "Total Purchase Amount": "sum"
})

In [9]:
rfm.columns = ["Recency", "Frequency", "Monetary"]
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
1,288,3,6290
2,72,6,16481
3,222,4,9423
4,441,5,7826
5,424,5,9769


In [10]:
rfm["R_Score"] = pd.qcut(rfm["Recency"], 4, labels=[4,3,2,1])
rfm["F_Score"] = pd.qcut(rfm["Frequency"], 4, labels=[1,2,3,4])
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 4, labels=[1,2,3,4])

In [11]:
rfm["RFM_Score"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str) + rfm["M_Score"].astype(str)
rfm.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
Customer ID,,,,,,,
1,288,3,6290,2,1,1,211
2,72,6,16481,4,3,3,433
3,222,4,9423,2,2,2,222
4,441,5,7826,1,2,1,121
5,424,5,9769,1,2,2,122


In [12]:
def segment(x):
    if x >= 9:
        return "Loyal / VIP"
    elif x >= 6:
        return "Potential Loyal"
    elif x >= 4:
        return "At Risk"
    else:
        return "Lost"

rfm["Segment_Score"] = rfm["R_Score"].astype(int) + rfm["F_Score"].astype(int) + rfm["M_Score"].astype(int)
rfm["Customer_Segment"] = rfm["Segment_Score"].apply(segment)

rfm.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment_Score,Customer_Segment
Customer ID,,,,,,,,,
1,288,3,6290,2,1,1,211,4,At Risk
2,72,6,16481,4,3,3,433,10,Loyal / VIP
3,222,4,9423,2,2,2,222,6,Potential Loyal
4,441,5,7826,1,2,1,121,4,At Risk
5,424,5,9769,1,2,2,122,5,At Risk


In [13]:
rfm.to_csv("customer_rfm_segment.csv")
from google.colab import files
files.download("customer_rfm_segment.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>